In [1]:
# !pip install requests pandas bokeh python-dotenv

import requests
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import HoverTool
import neo4j
from utils.neo4jdownloader import Neo4JDownloader
from pathlib import Path
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

output_notebook()


Loading BokehJS ...

In [6]:


def get_downloader():
    secrets_dir = Path("/secrets")
    if os.path.isdir(Path("/secrets")):
        NEO4J_URI = (secrets_dir / "neo4j_uri").read_text()
        NEO4J_USERNAME = (secrets_dir / "neo4j_user").read_text()
        NEO4J_PASSWORD = (secrets_dir / "neo4j_password").read_text()
        NEO4J_DATABASE = (secrets_dir / "neo4j_database").read_text()
    else:
        display("Using local env file to configure Tentris.")
        NEO4J_URI = os.getenv("NEO4J_URI")
        NEO4J_USERNAME = os.getenv("NEO4J_USER")
        NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
        NEO4J_DATABASE = os.getenv("NEO4J_DATABASE")

    return Neo4JDownloader(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD, NEO4J_DATABASE)

def connect_neo4j():
    downloader = get_downloader()
    return downloader

In [13]:
downloader = connect_neo4j()

orgs_query = """
MATCH (o:org)
RETURN o.name AS organization;
"""
organizations = downloader.run_custom_query(orgs_query)

print([org["organization"] for org in organizations])

'Using local env file to configure Tentris.'

['orcasound', 'tdwg', 'cbg-ethz', 'iobis', 'geco-bern', 'ub-unibe-ch', 'reymond-group', 'unicef', 'gbif', 'cgg-bern', 'digital-sustainability', 'DHBern', 'dis-unige', 'ethz-asl', 'carmonalab', 'GfellerLab', 'uzh-dqbm-cmi', 'ZurichNLP', 'VIS4ROB-lab', 'CMU-SAFARI', 'digitalmethodsinitiative', 'gold-lab', 'iastro-pt', '1like-1app', 'speech-separation-hse', 'AmenityDev', 'INRA-LPGP', 'stage-comprection', 'robinsonlabuzh', 'mie-lab', 'cvg', 'autonomousvision', 'fossasia', 'openscienceunil', 'aidos-lab', 'Paradigm-shift-AI', 'codeIndore', 'owncast', 'Azure', 'ecellietdavv', 'GDSC-IETDAVV', 'IET-DAVV', 'LedgerChirp', 'coastal-science', 'UWE-Ruby', 'MfN-Berlin', 'phylocom', 'phylotastic', 'asri-restor', 'ALA-herbarium', 'GBIF-Europe', 'gbif-norway', 'geco-nhm', 'uio-mana', 'geopython', 'sgrid', 'sci-wms', 'pyoceans', 'nco', 'ioos', 'xpublish-experiments', 'xpublish-community', 'cioos-siooc', 'cioos-atlantic', 'ocean-tracking-network', 'SustainableWebsites', 'Phoenix-Gaming-Club', 'FS-Stack', 

In [ ]:
ORGANIZATION_OF_INTEREST = "SwissDataScienceCenter"

repos_of_org_query = """
MATCH (o:org)-[:owner_of]->(repo:repo)
WHERE o.name = $org_name
RETURN repo.name AS repository
ORDER BY repository;
"""
parameters = {
    "org_name": ORGANIZATION_OF_INTEREST
}
repos_in_org = downloader.run_custom_query(repos_of_org_query, parameters)


print('repos for organization:', ORGANIZATION_OF_INTEREST)
for repo in repos_in_org:
    print(repo['repository'].split('/')[-1])

.github
amalthea
amalthea-jupyterhub-demo
avro4s
avrodisiac
best-practice-documentation
buildpack-workshop
calamus
compute-environment-template
contributed-project-templates
cookiefilter
datashim-charts
deepcloud-pidlrad
demo-shiny-serve
devcontainer-features
folioforge
homebrew-renku
jsonld4s
keycloak-theme
kube-manifest-backup
mlibra
mlschema-model-converters
mobiko_nlp
msamodeling
notebooks-cypress-tests
r10e-ds-py
rancher-pushprox
renga-authorization
renga-deployer
renga-documentation
renga-explorer
renga-graph
renga-home
renga-projects
renku
renku-actions
renku-admin-docs
renku-blog
renku-cli
renku-cli-plugin-example
renku-commons
renku-data-services
renku-demo
renku-design-docs
renku-dev-utils
renku-docs
renku-dummy-annotator
renku-dummy-format
renku-dummy-provider
renku-frontend-buildpacks
renku-gateway
renku-gateway-v2
renku-graph
renku-jena
renku-jupyterlab-ts
renku-jupyterlab-vnc
renku-maintenance-page
renku-mls
renku-notebooks
renku-on-aks
renku-outreach
renku-project-templa